# Carga de archivos excel a la base de datos con python
Se descargan todas las dependencias necesarias para leer los archivos CSV, XLSX y hacer la conexión con la base de datos en MySQL.

In [ ]:
!pip install pymysql
!pip install pandas
!pip install openpyxl
!pip install numpy

Una vez descargadas, se importan para:
- 1. Leer archivos csv a través de la libería que provee Python de forma natural (tomese en cuenta que este Jupyter Notebook se corre en Python 3.10.2)
- 2. Realizar la conexión y las operaciones de escritura dentro de la base de datos en MySQL.
- 3. Leer archivos Excel con extensión xlsx con la librería Pandas.

In [1]:
import csv
import pymysql
import pandas as pd

A continuación,se explica paso a paso el procesamiento de los archivos para llenar la base de datos.

Se define una función que contiene la conexión a la base de datos (host, puerto, usuario, contraseña y nombre de la base de datos), esto con el fin de que, si se migra a otro lado o cambian las variables de conexión, se pueda hacer el cambio más rápido, sin ir buscando la línea de conexión dentro de los procesamientos de los archivos.

In [7]:
def connection_mysql():
    conn = pymysql.connect(host="localhost", port=3306, user="root", passwd="", db="weather")
    return conn

Una vez definida la conexión (y el objeto `cursor` que permite la interacción), con `csv` se lee el archivo *stations.csv*, se itera en los registros y a su vez se va insertando en la base por medio de `commit()` ya que, por si sola la instrucción no se ejecuta, en caso de fallar, se realiza un `rollback()` para que no se ejecute la consulta, cuando termina de registrar se cierra la conexión a la base.

In [ ]:
#se abre el archivo csv de las estaciones
with open('../data/stations.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    #se inicia la conexion con MySQL
    conn = connection_mysql()
    cur = conn.cursor()
    #conforme se va leyendo las lineas del archivo, se van insertando en la base de datos
    for row in reader:
        sql = "INSERT INTO `stations` (`station_id`, `name`,`latitude`,`longitude`,`elevation`) VALUES (%s, %s,%s,%s,%s)"
        try:
            cur.execute(sql, (row['station_id'],row['name'],row['latitude'],row['longitude'],row['elevation']))
            conn.commit()
            print("Record was successfully")
        except:
            conn.rollback()
            print("Record was failed")
    cur.close()
    conn.close()


Se repite la misma acción para el archivo *variables.csv*

In [ ]:
#se abre el archivo csv de las variables
with open('../data/variables.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    #se inicia la conexion con MySQL
    conn = connection_mysql()
    cur = conn.cursor()
    #conforme se va leyendo las lineas del archivo, se van insertando en la base de datos
    for row in reader:
        sql = "INSERT INTO `variables` (`variable_id`, `name`, `description`, `units`) VALUES (%s, %s,%s,%s)"
        try:
            cur.execute(sql, (row['variable_id'],row['name'],row['description'],row['units']))
            conn.commit()
            print("Record was successfully")
        except:
            conn.rollback()
            print("Record was failed")
    cur.close()
    conn.close()

En esta sección, se usa `pandas` para la lectura de los archivos Excel.
Existen 5 variables que son cada una de las estaciones con sus respectivos archivos de observaciones de los años *2019*, *2020* y *2021* que se encuentran en una carpeta dentro del repositorio. Se trabaja una función que permite leer el archivo indicando donde se encuentran las columnas con los datos (segunda fila del archivo) debido a que cuando este se genera, toma la primera fila para poner el nombre de la tabla.

Una vez cargado el archivo, se limpia de celdas vacías sin modificarlo y se procede a realizar las inserciones en la base de datos.

In [9]:
#se definen las direcciones de los archivos
station1 = ["../step3/station1/2019.xlsx","../step3/station1/2020.xlsx","../step3/station1/2021.xlsx"]
station2 = ["../step3/station2/2019.xlsx","../step3/station2/2020.xlsx","../step3/station2/2021.xlsx"]
station3 = ["../step3/station3/2019.xlsx","../step3/station3/2020.xlsx","../step3/station3/2021.xlsx"]
station4 = ["../step3/station4/2019.xlsx","../step3/station4/2020.xlsx","../step3/station4/2021.xlsx"]
station5 = ["../step3/station5/2019.xlsx","../step3/station5/2020.xlsx","../step3/station5/2021.xlsx"]

def insert_records(station,paths):
    tam = len(paths)
    if tam > 0:
        for path in paths:       
            #se define el header a partir de la segunda fila, debido a que en la primera está el titulo del documento
            df = pd.read_excel(path,header=1)
            #Limpia la data de celdas vacias sin alterarla, después se manejan las columnas por separado para poder
            #procesar los registros de cada excel
            new_df = df.dropna()
            column_dates = new_df.columns[0]
            column_grades=new_df.columns[1]
            #se llama a la conexion de mysql
            conn = connection_mysql()
            cur = conn.cursor()
            #se iteran en los registros del excel en turno
            for index, row in new_df.iterrows():
                sql_observation = "INSERT INTO `observations` (`sensor_id`,`variable_id`, `observation_date`, `observed_value`, `status`) VALUES (%s,%s, %s,%s,%s)"
                try:
                    cur.execute(sql_observation, (station,"tmax",row[column_dates],row[column_grades],""))
                    conn.commit()
                    #print("Query success")
                except:
                    conn.rollback()
                    #print("Query fail")
            cur.close()
            conn.close()
            print("File ",path," was uploaded successfully")
    else:
        print("Array is not found")

insert_records(1,station1)
insert_records(2,station2)
insert_records(3,station3)
insert_records(4,station4)
insert_records(5,station5)

File  ../step3/station2/2019.xlsx  was uploaded successfully
File  ../step3/station2/2020.xlsx  was uploaded successfully
File  ../step3/station2/2021.xlsx  was uploaded successfully
File  ../step3/station3/2019.xlsx  was uploaded successfully
File  ../step3/station3/2020.xlsx  was uploaded successfully
File  ../step3/station3/2021.xlsx  was uploaded successfully
File  ../step3/station4/2019.xlsx  was uploaded successfully
File  ../step3/station4/2020.xlsx  was uploaded successfully
File  ../step3/station4/2021.xlsx  was uploaded successfully
File  ../step3/station5/2019.xlsx  was uploaded successfully
File  ../step3/station5/2020.xlsx  was uploaded successfully
File  ../step3/station5/2021.xlsx  was uploaded successfully
